## Homework 3 - Convolutional Neural Network
This is the example code of homework 3 of the machine learning course by Prof. Hung-yi Lee.

In this homework, you are required to build a convolutional neural network for image classification, possibly with some advanced training tips.

There are three levels here:

Easy: Build a simple convolutional neural network as the baseline. (2 pts)

Medium: Design a better architecture or adopt different data augmentations to improve the performance. (2 pts)

Hard: Utilize provided unlabeled data to obtain better results. (2 pts)

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

In [2]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

In [3]:
data_dir = "F:/Ai/Data/food-11/"

In [4]:
train_dataset = datasets.ImageFolder(os.path.join(data_dir, "training"), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, "validation"), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, "testing"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [5]:
# 简单的基本卷积神经网络

import torch.nn as nn
import torch.nn.functional as F
class FoodCNN(nn.Module):
    def __init__(self):
        super(FoodCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 11) # 11类

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FoodCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}")
        validate(model, val_loader)

def validate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    acc = 100 * correct / total
    print(f"Validation Accuracy: {acc}%")

train_model(model, criterion, optimizer, num_epochs=10)

Epoch 1/10, Loss: 0.6427682561010218
Validation Accuracy: 9.090909090909092%
Epoch 2/10, Loss: 0.6227598248176205
Validation Accuracy: 9.090909090909092%
Epoch 3/10, Loss: 0.6210234164227174
Validation Accuracy: 9.090909090909092%
Epoch 4/10, Loss: 0.6172049855722964
Validation Accuracy: 9.090909090909092%
Epoch 5/10, Loss: 0.613265186548233
Validation Accuracy: 9.090909090909092%
Epoch 6/10, Loss: 0.6080514005472745
Validation Accuracy: 8.484848484848484%
Epoch 7/10, Loss: 0.5955792497662664
Validation Accuracy: 7.424242424242424%
Epoch 8/10, Loss: 0.5732996522608699
Validation Accuracy: 7.878787878787879%
Epoch 9/10, Loss: 0.5179698761419956
Validation Accuracy: 7.7272727272727275%
Epoch 10/10, Loss: 0.39293053206116635
Validation Accuracy: 7.2727272727272725%


In [ ]:
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f"Test Accuracy: {100 * correct / total}%")

In [8]:
test_model(model, test_loader)

Test Accuracy: 26.85987451449059%


## Alexnet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_dir = "F:/Ai/Data/food-11/"

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(os.path.join(data_dir, "training"), transform=train_transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, "validation"), transform=test_transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, "evaluation"), transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32)


In [ ]:
model = models.alexnet(pretrained=True)

# 替换最后全连接层（原来是 1000 类）
model.classifier[6] = nn.Linear(4096, 11)
model = model.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
def evaluate(loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total


In [ ]:
num_epochs = 10
best_val_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}  Loss: {running_loss:.4f}  Train Acc: {train_acc*100:.2f}%  Val Acc: {val_acc*100:.2f}%")

    # 保存最佳模型
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_alexnet_food11.pth")
